To run this notebook, you need to already have a FitnessCalculatorConfiguration in Nexus. Therefore, I recomment to do the configuration and extraction through python before running this notebook:
```
python pipeline.py --step=configure_nexus --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
python pipeline.py --step=configure_model_from_gene --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
python pipeline.py --step=extract --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
```

## Loading the FitnessCalculatorConfiguration

In [9]:
species = "mouse"
brain_region = "SSCX"
data_access_point = "nexus"
nexus_project = "ncmv3"
nexus_organisation = "bbp"
nexus_endpoint = "prod"
forge_path = "./forge.yml"
forge_ontology_path = "./nsg.yml"

emodel = "L5_TPC:B_cAC"
ttype = "182_L4/5 IT CTX"
iteration_tag = "mettypesv12_1"

This next cell will ask for your Nexus token:

In [ ]:
from bluepyemodelnexus.nexus import NexusAccessPoint

access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project="ncmv3",
    organisation="bbp",
    endpoint="https://bbp.epfl.ch/nexus/v1",
    forge_path=forge_path,
    forge_ontology_path=forge_ontology_path,
    ttype=ttype,
    iteration_tag=iteration_tag
)

In [ ]:
fitness_config = access_point.get_fitness_calculator_configuration()

print(fitness_config)

## Adding EFeature or Protocol

In [12]:
from bluepyemodel.evaluation.efeature_configuration import EFeatureConfiguration
from bluepyemodel.evaluation.protocol_configuration import ProtocolConfiguration

In [14]:
protocol_name = "Step_1000"

stimuli = [{
    "amp": None,
    "delay": 700,
    "duration": 4,
    "holding_current": None,
    "thresh_perc": 1000,
    "totduration": 1000
}]

recordings = [
    {
        "type": "CompRecording",
        "location": "soma",
        "name": f"{protocol_name}.soma.v",
        "variable": "v"
    },
]

protocol_config = ProtocolConfiguration(
    name=protocol_name,
    stimuli=stimuli,
    recordings=recordings,
    validation=False
)

fitness_config.protocols.append(protocol_config)

In [16]:
efeature_def = EFeatureConfiguration(
    efel_feature_name="Spikecount",
    protocol_name=protocol_name,
    recording_name="soma.v",
    mean=1.0,
    std=0.1,
    efel_settings={"strict_stiminterval": False},
    threshold_efeature_std=None,
)

fitness_config.efeatures.append(efeature_def)

## Saving the FitnessCalculatorConfiguration

In [ ]:
access_point.store_fitness_calculator_configuration(fitness_config)

## Trying if the optimisation work:

```
python pipeline.py --step=test_optimise --emodel=EMODEL_NAME --iteration_tag=ITERATION_TAG --ttype=TTYPE
```